# Basic imports

In [1]:
import pandas as pd
import os
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
current_path = os.getcwd()
root_path=current_path.replace('\\forward_feature_selection','')

# Loading dataframe

In [3]:
mixed_df=pd.read_csv(root_path+"\\molecules.csv",sep="\t")

# Functions

In [4]:
def save_df_to_disk(df,name:str,separator="\t"):
    df.to_csv(name,sep=separator,index = False, header=True)

In [5]:
def get_data_and_true_prediction(df,not_wanted_features:list):
        temp_df=df.drop(not_wanted_features,axis=1)
        y=temp_df[temp_df.columns[-1]]
        x=temp_df.drop([temp_df.columns[-1]],axis=1)
        
        return x,y   

# Forward selection

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC

In [7]:
def get_roc_auc_score(x,y,model): # gets roc auc average
        cv_results = cross_validate(model, x, y, cv=10,scoring=('roc_auc'))
        roc_auc_avrg=cv_results['test_score'].mean()
        
        return roc_auc_avrg 

In [8]:
def get_best_score_index(score_lst:list):
    max_score = max(score_lst) # best score
    max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
    return max_score_index[0]

In [9]:
def show_results(df,roc,f_var,s_var):
    print("\n------------Results-------------\n")
    print("Best model whene first param is {} and second param is {}".format(f_var,s_var))
    print("Roc auc = {}".format(roc))
    print("\nFeatures\n")
    for cnt,col in enumerate(df.columns):
        print("{}. {}".format(cnt+1,col))
    print("")

In [10]:
def left_partition_is_better(first_score,middle_score,last_score):
    if ((first_score >= middle_score) and (last_score <= middle_score) and (first_score >= last_score)): return True               
    elif ((first_score <= middle_score) and (middle_score >= last_score) and(last_score <= first_score)): return True                   
    elif ((first_score >= middle_score) and (middle_score <= last_score) and (last_score >= first_score)): return False                  
    elif ((first_score >= middle_score) and (middle_score <= last_score) and (first_score >= last_score)): return True                
    else: return False

In [11]:
def forward_selection(x,y,model): # O(n) worst case scenario, where n depends on len(x.columns)
    
    def first_iteration(x,y,model):
        score_lst = []
        for i in range(len(x.columns)):
            k = x.columns[i]
            temp_x = x[[k]]
            score = get_roc_auc_score(temp_x,y,model)
            score_lst.append(score)
    
        max_score = max(score_lst) # best score
        max_score_index = [i for i, j in enumerate(score_lst) if j == max_score] # indx with best score
        top_score_index = max_score_index[0]
        new_feature = x.columns[top_score_index] 
        best_x = x[new_feature]
        new_x = x.drop(new_feature,axis=1)
        
        return best_x,new_x,max_score
    
    def else_iteration(best_x,x,y,model,actual_score):     
        new_x_lenght = len(x.columns)
        if (new_x_lenght > 0):
            score_lst = []
            for i in range(new_x_lenght):
                k = x.columns[i]
                temp_x = x[[k]]
                temp_new_x = pd.concat([best_x,temp_x],axis=1, ignore_index=True)
                score = get_roc_auc_score(temp_new_x,y,model)
                score_lst.append(score)

            max_score = max(score_lst) # best score

            if(max_score<actual_score):
                return best_x,actual_score # break condition, recursive function

            max_score_index = [i for i, j in enumerate(score_lst) if j == max_score] # indx with best score
            top_score_index = max_score_index[0]

            new_feature = x.columns[top_score_index]
            temp_x = x[new_feature]
            best_x = pd.concat([best_x,temp_x],axis=1)
            new_x = x.drop(new_feature,axis=1)

            return else_iteration(best_x,new_x,y,model,max_score)
        
        return best_x,actual_score
    
    f_best_x,new_x,f_score = first_iteration(x,y,model)
    best_x,best_score = else_iteration(f_best_x,new_x,y,model,f_score)
    
    return best_x,best_score

In [12]:
def inner_forward_partition(x,y,variable_prm_lst:list,static_prm): # (training data, prediction, variable list, static var)

    def else_iteration(x,y,variable_prm_lst:list,first_score,last_score,static_prm):       
        print("\ninner variable param list -> {}".format(variable_prm_lst))  
                           
        if (len(variable_prm_lst) == 2):
            print("\n---------inner param last iteration---------\n")
            first = variable_prm_lst[0]
            print("first is {}".format(first))
            last = variable_prm_lst[-1]
            print("last is {}".format(last))
            
            clf_first = SVC(C=static_prm,kernel='rbf',gamma=first,random_state=0) 
            first_df,first_score = forward_selection(x,y,clf_first)
            clf_last = SVC(C=static_prm,kernel='rbf',gamma=last,random_state=0)
            last_df,last_score = forward_selection(x,y,clf_last)

            print("left score: {} | right score: {}".format(first_score,last_score))
            
            if (last_score > first_score):
                print("last score is greater than first score")
                show_results(last_df,last_score,static_prm,last)
                return last_df,last_score,last

            print("first score is greater than last score")
            show_results(first_df,first_score,static_prm,first)
            return first_df,first_score,first
        else:        
            if(len(variable_prm_lst) % 2 == 0): 
                first = variable_prm_lst[0]
                print("first is {}".format(first))
                middle = variable_prm_lst[(len(variable_prm_lst)//2)-1]
                print("middle is {}".format(middle))
                last = variable_prm_lst[-1]
                print("last is {}".format(last))
   
                clf_middle = SVC(C=static_prm,kernel='rbf',gamma=middle,random_state=0)
                _,middle_score = forward_selection(x,y,clf_middle)
        
                print("left score: {} | middle score:{} | right score: {}".format(first_score,middle_score,last_score))

                if (left_partition_is_better(first_score,middle_score,last_score)):             
                    print("taken left: {}".format(variable_prm_lst[:(len(variable_prm_lst)//2)]))
                    return else_iteration(x,y,variable_prm_lst[:(len(variable_prm_lst)//2)],first_score,middle_score,static_prm)          
                
                print("taken right: {}".format(variable_prm_lst[(len(variable_prm_lst)//2)-1:]))
                return else_iteration(x,y,variable_prm_lst[(len(variable_prm_lst)//2)-1:],middle_score,last_score,static_prm)
            else:
                first = variable_prm_lst[0]
                print("first is {}".format(first))
                middle = variable_prm_lst[(len(variable_prm_lst)//2)]
                print("middle is {}".format(middle))
                last = variable_prm_lst[-1]
                print("last is {}".format(last))
    
                clf_middle = SVC(C=static_prm,kernel='rbf',gamma=middle,random_state=0)
                _,middle_score = forward_selection(x,y,clf_middle)      
                
                print("left score: {} | middle score:{} | right score: {}".format(first_score,middle_score,last_score))
                
                if (left_partition_is_better(first_score,middle_score,last_score)):
                    print("taken left: {}".format(variable_prm_lst[:(len(variable_prm_lst)//2)+1]))
                    return else_iteration(x,y,variable_prm_lst[:(len(variable_prm_lst)//2)+1],first_score,middle_score,static_prm)
            
                print("taken right: {}".format(variable_prm_lst[(len(variable_prm_lst)//2):]))
                return else_iteration(x,y,variable_prm_lst[(len(variable_prm_lst)//2):],middle_score,last_score,static_prm)
                   
    def first_iteration(x,y,variable_prm_lst:list,static_prm):
        print("\ninner variable param list -> {}".format(variable_prm_lst)) 
        
        if(len(variable_prm_lst) % 2 == 0):       
            first = variable_prm_lst[0]
            print("first is {}".format(first))
            middle = variable_prm_lst[(len(variable_prm_lst)//2)-1]
            print("middle is {}".format(middle))
            last = variable_prm_lst[-1]
            print("last is {}".format(last))
            
            clf_first = SVC(C=static_prm,kernel='rbf',gamma=first,random_state=0)
            _,first_score = forward_selection(x,y,clf_first)       
            clf_middle = SVC(C=static_prm,kernel='rbf',gamma=middle,random_state=0)
            _,middle_score = forward_selection(x,y,clf_middle)
            clf_last = SVC(C=static_prm,kernel='rbf',gamma=last,random_state=0)
            _,last_score = forward_selection(x,y,clf_last)
            
            if(left_partition_is_better(first_score,middle_score,last_score)):
                print("taken left: {}".format(variable_prm_lst[:(len(variable_prm_lst)//2)]))
                return variable_prm_lst[:(len(variable_prm_lst)//2)],first_score,middle_score
          
            print("taken right: {}".format(variable_prm_lst[(len(variable_prm_lst)//2)-1:]))
            return variable_prm_lst[(len(variable_prm_lst)//2)-1:],middle_score,last_score
        
        else:
            first = variable_prm_lst[0]
            print("first is {}".format(first))
            middle = variable_prm_lst[(len(variable_prm_lst)//2)]
            print("middle is {}".format(middle))
            last = variable_prm_lst[-1]
            print("last is {}".format(last))
            
            clf_first = SVC(C=static_prm,kernel='rbf',gamma=first,random_state=0)
            _,first_score = forward_selection(x,y,clf_first)       
            clf_middle = SVC(C=static_prm,kernel='rbf',gamma=middle,random_state=0)
            _,middle_score = forward_selection(x,y,clf_middle)
            clf_last = SVC(C=static_prm,kernel='rbf',gamma=last,random_state=0) 
            _,last_score = forward_selection(x,y,clf_last)
            
            if (left_partition_is_better(first_score,middle_score,last_score)):               
                print("taken left: {}".format(variable_prm_lst[:(len(variable_prm_lst)//2)+1]))
                return variable_prm_lst[:(len(variable_prm_lst)//2)+1],first_score,middle_score
         
            print("taken right: {}".format(variable_prm_lst[(len(variable_prm_lst)//2):]))
            return variable_prm_lst[(len(variable_prm_lst)//2):],middle_score,last_score

    if(len(variable_prm_lst) > 1 and len(variable_prm_lst) <=2):
        print("\nfirst param is {}".format(static_prm))
        best_df,best_score,best_variable_prm = else_iteration(x,y,variable_prm_lst,0,0,static_prm)
        return best_df,best_score,best_variable_prm                          
    else:
        print("\nouter param is {}".format(static_prm))
        new_variable_prm_lst,first_score,last_score = first_iteration(x,y,variable_prm_lst,static_prm)
        print("\nout prms from inner first iteration => variable_prm_lst: {} | left score: {} | right score: {}".format(new_variable_prm_lst,first_score,last_score))
        best_df,best_score,best_variable_prm = else_iteration(x,y,new_variable_prm_lst,first_score,last_score,static_prm)
        return best_df,best_score,best_variable_prm 

In [13]:
def forward_partition(x,y,K:list,P:list): # (training data, prediction, outer variable list, inner variable list)
    
    def first_iteration(x,y,K:list,P:list):
        print("outer param list -> {}".format(K))
        
        if(len(K) % 2 == 0):
            first = K[0]
            print("first is {}".format(first))
            middle = K[(len(K)//2)-1]
            print("middle is {}".format(middle))
            last = K[-1]
            print("last is {}".format(last))
            
            print("\n**************************")
            _,first_score,first_best_p = inner_forward_partition(x,y,P,first)
            print("\n**************************")
            _,middle_score,middle_best_p = inner_forward_partition(x,y,P,middle)
            print("\n**************************")
            _,last_score,last_best_p = inner_forward_partition(x,y,P,last)
            
            print("left score: {} | middle score:{} | right score: {}".format(first_score,middle_score,last_score))
            
            if (left_partition_is_better(first_score,middle_score,last_score)):          
                print("taken left: {}".format(K[:(len(K)//2)]))
                return K[:(len(K)//2)],first_score,middle_score
          
            print("taken right: {}".format(K[(len(K)//2)-1:]))
            return K[(len(K)//2)-1:],middle_score,last_score
        else:
            first = K[0]
            print("first is {}".format(first))
            middle = K[(len(K)//2)]
            print("middle is {}".format(middle))
            last = K[-1]
            print("last is {}".format(last))
            
            print("\n**************************")
            _,first_score,first_best_p = inner_forward_partition(x,y,P,first)
            print("\n**************************")
            _,middle_score,middle_best_p = inner_forward_partition(x,y,P,middle)
            print("\n**************************")
            _,last_score,last_best_p = inner_forward_partition(x,y,P,last)
            
            print("left score: {} | middle score:{} | right score: {}".format(first_score,middle_score,last_score))
            
            if (left_partition_is_better(first_score,middle_score,last_score)):
                print("taken left: {}".format(K[:(len(K)//2)+1]))
                return K[:(len(K)//2)+1],first_score,middle_score
         
            print("taken right: {}".format(K[(len(K)//2):]))
            return K[(len(K)//2):],middle_score,last_score

    def else_iteration(x,y,K:list,P:list,first_score,last_score):
        
        if(len(K) == 2):          
            print("\n---------outer param last iteration---------\n")
            first = K[0]
            print("first is {}".format(first))
            last = K[-1]
            print("last is {}".format(last))
            print("\n**************************")
            first_df,first_score,first_best_p = inner_forward_partition(x,y,P,first)
            print("\n**************************")
            last_df,last_score,last_best_p = inner_forward_partition(x,y,P,last) 
            print("")
            
            if (last_score > first_score):
                print("last score is greater than first score")
                return last_df,last_score,last,last_best_p

            print("first score is greater than last score")
            return first_df,first_score,first,first_best_p        
        else:
            print("first param list -> {}".format(K))  
        
            if(len(K) % 2 == 0):
                first = K[0]
                print("first is {}".format(first))
                middle = K[(len(K)//2)-1]
                print("middle is {}".format(middle))
                last = K[-1]
                print("last is {}".format(last))
                
                print("\n**************************")
                _,middle_score,middle_best_p = inner_forward_partition(x,y,P,middle)
                
                print("left score: {} | middle score:{} | right score: {}".format(first_score,middle_score,last_score))
                
                if (left_partition_is_better(first_score,middle_score,last_score)):                  
                    print("taken left: {}".format(K[:(len(K)//2)]))
                    return else_iteration(x,y,K[:(len(K)//2)],P,first_score,middle_score)
                
                print("taken right: {}".format(K[(len(K)//2)-1:]))
                return else_iteration(x,y,K[(len(K)//2)-1:],P,middle_score,last_score)
            else:
                first = K[0]
                print("first is {}".format(first))
                middle = K[(len(K)//2)]
                print("middle is {}".format(middle))
                last = K[-1]
                print("last is {}".format(last))
                
                print("\n**************************")
                _,middle_score,middle_best_p = inner_forward_partition(x,y,P,middle)
                
                print("left score: {} | middle score:{} | right score: {}".format(first_score,middle_score,last_score))

                if (left_partition_is_better(first_score,middle_score,last_score)):                    
                    print("taken left: {}".format(K[:(len(K)//2)+1]))
                    return else_iteration(x,y,K[:(len(K)//2)+1],P,first_score,middle_score)
               
                print("taken right: {}".format(K[(len(K)//2):]))
                return else_iteration(x,y,K[(len(K)//2):],P,middle_score,last_score)
    
    if(len(K) > 1 and len(K) <= 2):
        best_df,best_score,best_first_var,best_second_var = else_iteration(x,y,K,P,0,0)
        show_results(best_df,best_score,best_first_var,best_second_var)
        return [best_df,best_score,best_first_var,best_second_var]                          
    else:
        new_lst,first_score,last_score = first_iteration(x,y,K,P)
        print("\nout prms from outer first iteration => lst: {} | left score: {} | right score: {}".format(new_lst,first_score,last_score))
        best_df,best_score,best_first_var,best_second_var = else_iteration(x,y,new_lst,P,first_score,last_score)
        show_results(best_df,best_score,best_first_var,best_second_var)
        return [best_df,best_score,best_first_var,best_second_var]  

In [14]:
def guarantee_results(x,y,K:list,P:list):
    best_outer_prm_lst = []
    for i in K:
        print("First param is {}\n".format(i))
        best_inner_prm_lst = []
        for j in P:
            print("Second param is {}".format(j))
            clf = SVC(random_state=0, C=i,gamma=j) # linear model
            best_df,best_score = forward_selection(x,y,clf)
            print("ROC AUC = {}".format(best_score))
            print("\nFeatures\n")
            for c,d in enumerate(best_df.columns):
                print("{}. {}".format(c+1,d))       
            print("")
            best_inner_prm_lst.append([best_df,i,j,best_score])
            
        score_lst = [d  for [a,b,c,d] in best_inner_prm_lst]
        index = get_best_score_index(score_lst)
        best_outer_prm_lst.append([best_inner_prm_lst[index][0],best_inner_prm_lst[index][1],best_inner_prm_lst[index][2],best_inner_prm_lst[index][3]])
        
    score_lst = [d  for [a,b,c,d] in best_outer_prm_lst]
    index = get_best_score_index(score_lst)
    print("Final results")
    print("Best model when first param = {} ,second param = {} ,ROC AUC = {}".format(best_outer_prm_lst[index][1],best_outer_prm_lst[index][2],best_outer_prm_lst[index][3]))
    return best_outer_prm_lst[index][0]

# Data

In [15]:
unnecesary_features=["m_name"]
x,y = get_data_and_true_prediction(mixed_df,unnecesary_features)

In [16]:
C = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
Gamma = [0.000001,0.00001,0.0001,0.001,0.01,0.1,1,5,10,15,20]

In [17]:
start = time.time()

best_data = forward_partition(x,y,C,Gamma)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

outer param list -> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
first is 1
middle is 15
last is 30

**************************

outer param is 1

inner variable param list -> [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 15, 20]
first is 1e-06
middle is 0.1
last is 20
taken left: [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]

out prms from inner first iteration => variable_prm_lst: [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1] | left score: 0.8697986606626695 | right score: 0.8931531101861486

inner variable param list -> [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]
first is 1e-06
middle is 0.0001
last is 0.1
left score: 0.8697986606626695 | middle score:0.8670498395768297 | right score: 0.8931531101861486
taken right: [0.0001, 0.001, 0.01, 0.1]

inner variable param list -> [0.0001, 0.001, 0.01, 0.1]
first is 0.0001
middle is 0.001
last is 0.1
left score: 0.8670498395768297 | middle score:0.8635243494293311 | right score: 0.89

left score: 0.8926790411360519 | middle score:0.902148318137522 | right score: 0.8846902201740912
taken left: [0.001, 0.01]

inner variable param list -> [0.001, 0.01]

---------inner param last iteration---------

first is 0.001
last is 0.01
left score: 0.8926790411360519 | right score: 0.902148318137522
last score is greater than first score

------------Results-------------

Best model whene first param is 18 and second param is 0.01
Roc auc = 0.902148318137522

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_HBA
4. m_logp
5. n_atoms_stereo_centers
6. n_O
7. n_hetero_atoms
8. n_primary_carbon_atoms
9. n_HOH
10. n_hetero_cycles
11. n_amide_bonds
12. n_aliphatic_carbocycles
13. n_saturated_carbocycles
14. n_briged_head_atoms
15. fraction_CSP3
16. n_aromatic_rings

left score: 0.9018693583486117 | middle score:0.902148318137522 | right score: 0.8916647310589167
taken left: [15, 16, 17, 18]
first param list -> [15, 16, 17, 18]
first is 15
middle is 16
last is 18

*******************

'01:36:43'

In [18]:
start = time.time()

g_data = guarantee_results(x,y,C,Gamma)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

First param is 1

Second param is 1e-06
ROC AUC = 0.8697986606626695

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_briged_head_atoms
7. n_hetero_cycles
8. n_amide_bonds
9. n_aliphatic_carbocycles
10. n_aromatic_heterocycles

Second param is 1e-05
ROC AUC = 0.8672450262623259

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_briged_head_atoms
7. n_amide_bonds
8. n_saturated_carbocycles
9. n_hetero_cycles
10. n_aromatic_heterocycles
11. fraction_CSP3

Second param is 0.0001
ROC AUC = 0.8670498395768297

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_carbocycles
9. n_hetero_cycles
10. n_aromatic_heterocycles
11. n_atoms_without_Hydrogen

Second param is 0.001
ROC AUC = 0.8635243494293311

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_HBA
4. m_logp
5. n_briged_head_atoms
6. n_saturated_carbocy

ROC AUC = 0.877773054894868

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_HBA
4. m_logp
5. n_aromatic_carbocycles
6. n_O
7. n_amide_bonds
8. n_briged_head_atoms
9. n_saturated_carbocycles
10. n_rings
11. n_strict_rotable_bonds
12. n_non_strict_rotable_bonds

Second param is 0.01
ROC AUC = 0.8863947937829311

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_hetero_atoms
4. n_atoms_without_Hydrogen
5. n_atoms_stereo_centers
6. m_logp
7. n_O
8. n_amide_bonds
9. n_atoms_with_Hydrogen
10. n_briged_head_atoms
11. n_hetero_cycles
12. n_aromatic_carbocycles
13. fraction_CSP3
14. n_aromatic_rings

Second param is 0.1
ROC AUC = 0.8996147108253636

Features

1. n_O
2. n_aliphatic_heterocycles
3. m_logp
4. n_aromatic_carbocycles
5. n_aromatic_rings
6. fraction_CSP3
7. n_HBD
8. n_amide_bonds
9. n_strict_rotable_bonds
10. n_HOH
11. n_rings
12. n_saturated_rings
13. n_briged_head_atoms
14. n_saturated_carbocycles
15. n_hetero_cycles
16. n_saturated_heterocycles

Second param is 1
ROC AUC = 

ROC AUC = 0.8880108714054474

Features

1. n_O
2. n_aliphatic_heterocycles
3. n_non_strict_rotable_bonds
4. n_aromatic_carbocycles
5. n_amide_bonds
6. n_saturated_carbocycles
7. n_briged_head_atoms
8. n_HBD
9. fraction_CSP3
10. n_aromatic_heterocycles

Second param is 1
ROC AUC = 0.8269329753233239

Features

1. n_HBD
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_O
5. n_saturated_carbocycles
6. n_aliphatic_rings

Second param is 5
ROC AUC = 0.810828009121616

Features

1. n_HBA
2. n_aliphatic_heterocycles
3. fraction_CSP3
4. n_HBD
5. n_HOH

Second param is 10
ROC AUC = 0.8011274915142955

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA
5. n_HBD
6. n_saturated_carbocycles

Second param is 15
ROC AUC = 0.7939947124357895

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA
5. n_HBD
6. n_saturated_carbocycles

Second param is 20
ROC AUC = 0.7864494424417355

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA
5. n_satu

ROC AUC = 0.8163537305097204

Features

1. n_HBA
2. n_aliphatic_heterocycles
3. fraction_CSP3
4. n_HOH
5. n_HBD

Second param is 10
ROC AUC = 0.798541235671341

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA
5. n_HBD
6. n_saturated_carbocycles

Second param is 15
ROC AUC = 0.7887928569363922

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA
5. n_HBD
6. n_saturated_carbocycles

Second param is 20
ROC AUC = 0.7878723076986605

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_O
5. n_saturated_rings

First param is 11

Second param is 1e-06
ROC AUC = 0.8660105472556696

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_carbocycles

Second param is 1e-05
ROC AUC = 0.8663775901012505

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_ca

ROC AUC = 0.8666377099747287

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_carbocycles
9. n_hetero_cycles
10. n_aromatic_heterocycles

Second param is 1e-05
ROC AUC = 0.8662279805262376

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_carbocycles
9. n_hetero_cycles
10. n_aromatic_heterocycles
11. n_atoms_without_Hydrogen
12. fraction_CSP3

Second param is 0.0001
ROC AUC = 0.8710293200700328

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_hetero_cycles
9. n_aromatic_heterocycles
10. n_saturated_carbocycles
11. n_aliphatic_rings
12. n_aromatic_rings
13. n_saturated_rings

Second param is 0.001
ROC AUC = 0.8852435975257255

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_O
4. m_logp
5. n_aromatic_carbocyc

...

ROC AUC = 0.8913868035941398

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_O
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. m_mr
8. n_primary_carbon_atoms
9. n_hetero_cycles
10. n_HOH
11. n_briged_head_atoms
12. n_saturated_heterocycles
13. n_atoms_without_Hydrogen
14. fraction_CSP3
15. n_rings

Second param is 0.01
ROC AUC = 0.9014622871347637

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_HBA
4. m_logp
5. n_atoms_stereo_centers
6. n_O
7. n_hetero_atoms
8. n_briged_head_atoms
9. n_saturated_carbocycles
10. n_aromatic_carbocycles
11. n_amide_bonds
12. fraction_CSP3
13. n_HOH
14. n_aliphatic_rings
15. n_aliphatic_carbocycles

Second param is 0.1
ROC AUC = 0.8852945943379085

Features

1. n_O
2. n_aliphatic_heterocycles
3. n_non_strict_rotable_bonds
4. n_aromatic_carbocycles
5. n_saturated_carbocycles
6. n_briged_head_atoms
7. n_HBD
8. n_amide_bonds
9. fraction_CSP3
10. n_aromatic_heterocycles

Second param is 1
ROC AUC = 0.8463607219249129

Features

1. n_HBD
2. fr

ROC AUC = 0.8751534291743059

Features

1. n_O
2. n_aliphatic_heterocycles
3. n_non_strict_rotable_bonds
4. n_aromatic_carbocycles
5. n_amide_bonds
6. n_briged_head_atoms
7. n_saturated_rings

Second param is 1
ROC AUC = 0.8225892134515963

Features

1. n_aliphatic_rings
2. n_O
3. fraction_CSP3
4. n_briged_head_atoms

Second param is 5
ROC AUC = 0.8028185029648348

Features

1. n_aliphatic_rings
2. n_O
3. fraction_CSP3
4. n_briged_head_atoms
5. n_saturated_carbocycles

Second param is 10
ROC AUC = 0.784086220227772

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA

Second param is 15
ROC AUC = 0.7903879396668484

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA
5. n_HBD
6. n_saturated_rings
7. n_saturated_heterocycles

Second param is 20
ROC AUC = 0.7832574704135904

Features

1. n_HBD
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_briged_head_atoms

First param is 24

Second param is 1e-06
ROC AUC = 0.8669897841203772

Features

1. n_

ROC AUC = 0.8039464719290421

Features

1. n_aliphatic_rings
2. n_O
3. fraction_CSP3
4. n_saturated_carbocycles
5. n_briged_head_atoms

Second param is 10
ROC AUC = 0.7853127374345507

Features

1. n_HOH
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_briged_head_atoms

Second param is 15
ROC AUC = 0.8039310515873016

Features

1. n_aliphatic_rings
2. n_O
3. n_saturated_carbocycles
4. fraction_CSP3
5. n_briged_head_atoms

Second param is 20
ROC AUC = 0.7814630846051568

Features

1. n_HBD
2. fraction_CSP3
3. n_aliphatic_heterocycles

First param is 27

Second param is 1e-06
ROC AUC = 0.8669410068050807

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_carbocycles
9. n_hetero_cycles
10. n_aromatic_heterocycles

Second param is 1e-05
ROC AUC = 0.8663832420758799

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms


ROC AUC = 0.7821103002205044

Features

1. n_HBD
2. fraction_CSP3
3. n_aliphatic_heterocycles

First param is 30

Second param is 1e-06
ROC AUC = 0.8668369485324481

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_carbocycles
9. n_hetero_cycles
10. n_aromatic_heterocycles

Second param is 1e-05
ROC AUC = 0.8663824420246767

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_saturated_carbocycles
9. n_hetero_cycles
10. n_aromatic_heterocycles
11. n_atoms_without_Hydrogen

Second param is 0.0001
ROC AUC = 0.8749574682457097

Features

1. n_HBD
2. n_O
3. n_aliphatic_heterocycles
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. n_briged_head_atoms
8. n_hetero_cycles
9. n_saturated_rings
10. n_aliphatic_rings
11. n_saturated_carbocycles
12. n_rings

Second param is 0.001
ROC AUC = 0.896112860900517

Featur

'06:16:24'

In [19]:
#new_df=get_df_with_name_and_prediction(best_df,y,mixed_df)
#save_df_to_disk(new_df,"best_classifier_SVC.csv")

C = 17
Gamma = 0.01
Tol = 0.001
ROC AUC = 0.9022

C = 29
Gamma = 0.01
Tol = 0.001
ROC AUC = 0.9051